In [ ]:
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Markdown

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys.msq import (
    mosaiq_connect, delivery_data_from_mosaiq, get_patient_fields,
    get_qcls_by_date, multi_mosaiq_connect, get_incomplete_qcls_across_sites,
    get_recently_completed_qcls_across_sites
)
from pymedphys.coll import calc_mu_density
from pymedphys.type import get_delivery_parameters

In [ ]:
qa_centre = 'rccc'
treatment_centres = ['nbcc', 'sash']

centres = [qa_centre] + treatment_centres

qa_patient_ids = ['999999', '999998', '999997', '999996']

servers = {
    'rccc': 'msqsql', 
    'nbcc': 'physics-server:31433',  # nbccc-msq  or  physics-server:31433
    'sash': 'physics-server'
}

servers_list = [
    item for _, item in servers.items()
]

physics_location = {
    'rccc': 'Physics_Check',
    'nbcc': 'Physics',
    'sash': 'Physics_Check'
}

imrt_task_names = {
    'nbcc': 'Physics Check IMRT',
    'rccc': 'IMRT Physics Check',
    'sash': 'Physics QA           '
}

comparison_field_types = [
    'DMLC', 'VMAT'
]

In [ ]:
with multi_mosaiq_connect(servers_list) as cursors:
    incomplete_qcls = get_incomplete_qcls_across_sites(cursors, servers, centres, physics_location)
    recently_completed_qcls = get_recently_completed_qcls_across_sites(cursors, servers, centres, physics_location)
    
display(incomplete_qcls)
display(recently_completed_qcls)

In [ ]:
plans_to_verify = pd.DataFrame()

for centre in treatment_centres:
    plans_to_verify = plans_to_verify.append(incomplete_qcls[(
        (incomplete_qcls['centre'] == centre) &
        (incomplete_qcls['task'] == imrt_task_names[centre])
    )], sort=False)
    
    plans_to_verify = plans_to_verify.append(recently_completed_qcls[(
        (recently_completed_qcls['centre'] == centre) &
        (recently_completed_qcls['task'] == imrt_task_names[centre])
    )], sort=False)

plans_to_verify

In [ ]:
patient_ids = plans_to_verify['patient_id'].values
patient_ids

In [ ]:
all_qa_fields = pd.DataFrame()

with mosaiq_connect(servers[qa_centre]) as cursor:
    for qa_patient_id in qa_patient_ids:
        qa_fields = get_patient_fields(cursor, qa_patient_id)
        qa_fields['patient_id'] = [qa_patient_id] * len(qa_fields)
        
        all_qa_fields = all_qa_fields.append(qa_fields)
        
# all_qa_fields

In [ ]:
clinical_field_ids = {}
clinical_field_tables = {}

with multi_mosaiq_connect(servers_list) as cursors:
    for qcl in plans_to_verify.itertuples():
        patient_id = qcl.patient_id
        centre = qcl.centre
        
        display(Markdown('### Centre: {}, Patient ID: {}'.format(centre, patient_id)))
        
        cursor = cursors[servers[centre]]

        clinical_fields = get_patient_fields(cursor, patient_id)
        clinical_beam_reference = (
            (clinical_fields['field_version'] == 0) & 
            (clinical_fields['monitor_units'] != 0) &
            (clinical_fields['field_type'].isin(comparison_field_types))
        )
        clinical_fields = clinical_fields[clinical_beam_reference]
        
        clinical_field_tables[(centre, patient_id)] = clinical_fields
        
        display(clinical_fields)
        
        clinical_field_ids[(centre, patient_id)] = clinical_fields['field_id'].values.astype(str)


clinical_field_ids

In [ ]:
centre_patient_id_tuples = list(clinical_field_ids.keys())
centre_patient_id_tuples

In [ ]:
qa_field_site_agreeing = {
    (centre, patient_id): all_qa_fields[(
        (all_qa_fields['site'] == patient_id)
    )]
    for centre, patient_id in centre_patient_id_tuples
}

for centre, patient_id in centre_patient_id_tuples:
    display(Markdown('### Centre: {}, Patient ID: {}'.format(centre, patient_id)))
    display(qa_field_site_agreeing[(centre, patient_id)])

In [ ]:
qa_field_ids = {
    key: qa_field_site_agreeing[key]['field_id'].values.astype(str)
    for key in centre_patient_id_tuples
}

qa_field_ids

In [ ]:
def determine_mu_density(cursor, field_id):

    delivery_data = delivery_data_from_mosaiq(cursor, field_id)
        
    mu, mlc, jaw = get_delivery_parameters(delivery_data)
    mu_density = calc_mu_density(mu, mlc, jaw)
    
    return mu_density

In [ ]:
with multi_mosaiq_connect(servers_list) as cursors:

    for centre, patient_id in centre_patient_id_tuples:
        display(Markdown('### Centre: {}, Patient ID: {}'.format(centre, patient_id)))
        
        display(Markdown('#### Clinical Fields'))
        display(clinical_field_tables[(centre, patient_id)])
        
        display(Markdown('#### QA Fields'))
        display(qa_field_site_agreeing[(centre, patient_id)])
        
        clinical_cursor = cursors[servers[centre]]
        qa_cursor = cursors[servers[qa_centre]]
        
        key = (centre, patient_id)
        
        display(Markdown('#### Comparison'))
        
        if len(clinical_field_ids[key]) == 0 or len(qa_field_ids[key]) == 0:
            print('missing fields')
            continue
        
        clinical_mu_density = np.sum([
            determine_mu_density(clinical_cursor, field_id)
            for field_id in clinical_field_ids[key]
        ], axis=0)

        qa_mu_density = np.sum([
            determine_mu_density(qa_cursor, field_id)
            for field_id in qa_field_ids[key]
        ], axis=0)

        plt.figure()
        plt.pcolormesh(clinical_mu_density)
        plt.colorbar()
        plt.title('Clinical MU density')
        plt.xlabel('MLC direction (mm)')
        plt.ylabel('Jaw direction (mm)')
        plt.gca().invert_yaxis()

        plt.figure()
        plt.pcolormesh(qa_mu_density)
        plt.colorbar()
        plt.title('QA MU density')
        plt.xlabel('MLC direction (mm)')
        plt.ylabel('Jaw direction (mm)')
        plt.gca().invert_yaxis()
        
        plt.show()
        
        # Verify that every pixel agrees within 0.1 MU
        maximum_deviation = np.max(np.abs(clinical_mu_density - qa_mu_density))
        print("Maximum MU Density deviation between fields: {}".format(maximum_deviation))